In [4]:
import time
import json
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup


# --- 1단계: 모든 상품의 고유 ID만 수집하는 함수 (수정된 버전) ---
def get_all_product_ids():
    """
    '버거' 카테고리 페이지 URL에 직접 접속하여 클릭 상호작용 없이 ID를 수집합니다.
    """
    driver = webdriver.Chrome()
    # 결정적 수정: '버거' 탭이 처음부터 선택된 URL로 직접 접속
    burger_page_url = "https://www.lotteeatz.com/brand/ria?categoryId=C1001"
    driver.get(burger_page_url)
    driver.maximize_window()
    product_ids = []

    try:
        print("✅ 1단계: '버거' 카테고리 페이지에 직접 접속했습니다.")

        # 팝업은 여전히 나타날 수 있으므로 제거 로직은 유지합니다.
        try:
            popup_close_button = WebDriverWait(driver, 3).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button.btn-close-popup")))
            popup_close_button.click()
            print("✅ 1단계: 프로모션 팝업을 닫았습니다.")
        except TimeoutException:
            print("ℹ️ 1단계: 프로모션 팝업이 없습니다.")

        # 분기점: [페이지 로딩] -> [상품 목록의 '내용물'이 채워질 때까지 대기]
        # li 태그 중 onclick 속성에 'goBrandDetail' 문자열이 포함된 첫 번째 요소가 나타날 때까지 기다립니다.
        print("⏳ 상품 목록의 최종 렌더링을 기다립니다...")
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, 'ul.prod-list > li[onclick*="goBrandDetail"]'))
        )
        print("✅ 상품 목록 렌더링 완료!")

        # 렌더링이 막 끝난 시점의 DOM 불안정성을 해소하기 위해 아주 짧은 추가 대기
        time.sleep(1)

        soup = BeautifulSoup(driver.page_source, 'html.parser')

        # 이제 이 선택자는 반드시 성공해야 합니다.
        product_list = soup.select('ul.prod-list > li')

        if not product_list:
            print("⚠️ 1단계: 'ul.prod-list > li' 선택자로도 상품 목록을 찾을 수 없습니다.")

        for item in product_list:
            onclick_attr = item.get('onclick')
            if onclick_attr and 'goBrandDetail' in onclick_attr:
                product_id = onclick_attr.split("'")[1]
                product_ids.append(product_id)

        if not product_ids:
             print("⚠️ 1단계: ID를 수집하지 못했습니다. 사이트 구조를 확인해주세요.")
        else:
            print(f"✅ 1단계: 총 {len(product_ids)}개의 상품 ID 수집을 완료했습니다.")

    except Exception as e:
        print(f"❌ 1단계: ID 수집 중 오류가 발생했습니다: {e}")
    finally:
        driver.quit()
        print("✅ 1단계: 브라우저를 종료합니다.")
        return product_ids

    # --- 2단계: ID 리스트를 기반으로 상세 정보를 수집하는 함수 ---
def get_product_details(product_ids):
    """
    1단계에서 수집한 ID 리스트를 받아, 각 상세 페이지를 직접 방문하여
    이름, 영양정보, 알러지정보, 원산지정보를 수집합니다.
    """
    if not product_ids:
        print("⚠️ 2단계: 수집할 ID가 없어 작업을 종료합니다.")
        return []

    driver = webdriver.Chrome()
    all_products_data = []

    for i, pid in enumerate(product_ids):
        try:
            # 2-1. 상세 페이지 URL을 직접 만들어 접속 (가장 안정적인 방법)
            detail_url = f"https://www.lotteeatz.com/products/introductions/{pid}"
            driver.get(detail_url)

            # 2-2. '상세정보' 버튼을 JS로 클릭 (단일 테스트에서 성공한 로직)
            detail_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, "button.btn-fold.detail-info"))
            )
            driver.execute_script("arguments[0].click();", detail_button)

            # 2-3. 상세정보 컨텐츠가 로드될 때까지 대기
            WebDriverWait(driver, 5).until(
                EC.presence_of_element_located((By.XPATH, "//div[@class='btext-tit' and text()='영양소 정보']"))
            )

            # 2-4. BeautifulSoup으로 페이지를 파싱하여 데이터 추출
            detail_soup = BeautifulSoup(driver.page_source, 'html.parser')

            name = detail_soup.select_one('div.prod-tit').text.strip()

            nutrition_info, allergy_info, origin_info = {}, "", {}

            # 영양소 정보 추출
            nutrition_div = detail_soup.find('div', class_='btext-tit', string='영양소 정보')
            if nutrition_div:
                nutrition_table = nutrition_div.find_next_sibling('div', class_='tbl-info-wrap')
                if nutrition_table:
                    for row in nutrition_table.select('tbody > tr'):
                        key = row.select_one('th').text.strip()
                        value = row.select_one('td').text.strip()
                        nutrition_info[key] = value

            # 알러지 정보 추출
            allergy_p = detail_soup.find('div', class_='btext-tit', string='알러지 정보')
            if allergy_p:
                allergy_info = allergy_p.find_next_sibling('p', class_='btext').text.strip()

            # 원산지 정보 추출
            origin_div = detail_soup.find('div', class_='btext-tit', string='원산지 정보')
            if origin_div:
                origin_table = origin_div.find_next_sibling('div', class_='tbl-info-wrap')
                if origin_table:
                    for row in origin_table.select('tbody > tr'):
                        key = row.select_one('th').text.strip()
                        value = row.select_one('td').text.strip()
                        origin_info[key] = value

            # 2-5. 추출된 데이터를 딕셔너리로 정리
            product_data = {
                "id": pid,
                "name": name,
                "nutrition": nutrition_info,
                "allergy": allergy_info,
                "origin": origin_info
            }
            all_products_data.append(product_data)
            print(f"({i+1}/{len(product_ids)}) ✅ '{name}' 정보 수집 성공")

        except Exception as e:
            print(f"({i+1}/{len(product_ids)}) ❌ ID '{pid}' 처리 중 오류 발생: {e}")

    driver.quit()
    print("✅ 2단계: 브라우저를 종료합니다.")
    return all_products_data

# --- 3단계: 수집된 데이터를 파일로 저장하는 함수 ---
def save_data(data):
    """수집된 데이터를 JSON과 CSV 파일로 저장합니다."""
    # JSON 저장
    with open("lotte_eatz_burgers_final.json", "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=4)
    print("✅ JSON 파일 저장 완료: lotte_eatz_burgers_final.json")

    # CSV 저장을 위해 데이터 평탄화
    csv_data = []
    for item in data:
        nutrition_str = ", ".join([f"{k}: {v}" for k, v in item['nutrition'].items()])
        origin_str = ", ".join([f"{k}: {v}" for k, v in item['origin'].items()])

        csv_data.append([
            item['id'],
            item['name'],
            nutrition_str,
            item['allergy'],
            origin_str
        ])

    # CSV 저장
    with open("lotte_eatz_burgers_final.csv", "w", encoding="utf-8-sig", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(['ID', '이름', '영양정보', '알러지정보', '원산지정보'])
        writer.writerows(csv_data)
    print("✅ CSV 파일 저장 완료: lotte_eatz_burgers_final.csv")

    # --- 메인 실행 블록 ---
if __name__ == '__main__':
    ids = get_all_product_ids()

    if ids:
        # details = get_product_details(ids) # 2단계 실행
        # if details:
        #     save_data(details) # 3단계 실행
        print("\n🎉 1단계 ID 수집이 성공적으로 완료되었습니다!")
        print("이제 2단계 상세 정보 수집을 진행할 수 있습니다.")
    else:
        print("\n최종적으로 수집된 데이터가 없습니다.")

✅ 1단계: '버거' 카테고리 페이지에 직접 접속했습니다.
ℹ️ 1단계: 프로모션 팝업이 없습니다.
⏳ 상품 목록의 최종 렌더링을 기다립니다...
❌ 1단계: ID 수집 중 오류가 발생했습니다: Message: 
Stacktrace:
	GetHandleVerifier [0x0x7ff622e4fca5+79861]
	GetHandleVerifier [0x0x7ff622e4fd00+79952]
	(No symbol) [0x0x7ff622bccada]
	(No symbol) [0x0x7ff622c24457]
	(No symbol) [0x0x7ff622c2471c]
	(No symbol) [0x0x7ff622c78217]
	(No symbol) [0x0x7ff622c4cb1f]
	(No symbol) [0x0x7ff622c74f8b]
	(No symbol) [0x0x7ff622c4c8b3]
	(No symbol) [0x0x7ff622c15272]
	(No symbol) [0x0x7ff622c16043]
	GetHandleVerifier [0x0x7ff62310b9dd+2946349]
	GetHandleVerifier [0x0x7ff623105c5a+2922410]
	GetHandleVerifier [0x0x7ff6231259e7+3052855]
	GetHandleVerifier [0x0x7ff622e6aa8e+189918]
	GetHandleVerifier [0x0x7ff622e72a2f+222591]
	GetHandleVerifier [0x0x7ff622e58ac4+116244]
	GetHandleVerifier [0x0x7ff622e58c79+116681]
	GetHandleVerifier [0x0x7ff622e3f058+11176]
	BaseThreadInitThunk [0x0x7ffc307c7374+20]
	RtlUserThreadStart [0x0x7ffc31d7cc91+33]

✅ 1단계: 브라우저를 종료합니다.

최종적으로 수집된 데이터가 없습니다.
